In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"]='1'

from pyspark.sql import SparkSession
spark = (SparkSession
    .builder
    .master("local")
    .appName("trade")
    .getOrCreate()
)

import pyspark.sql.functions as F

In [2]:
data22 = spark.read.csv(".//archive/minute_220301_230228.csv", sep=';', encoding="UTF-8", header=True, inferSchema=True)
data22.printSchema()

root
 |-- <TICKER>: string (nullable = true)
 |-- <PER>: integer (nullable = true)
 |-- <DATE>: integer (nullable = true)
 |-- <TIME>: integer (nullable = true)
 |-- <OPEN>: double (nullable = true)
 |-- <HIGH>: double (nullable = true)
 |-- <LOW>: double (nullable = true)
 |-- <CLOSE>: double (nullable = true)
 |-- <VOL>: integer (nullable = true)



In [3]:
data22 = data22.drop('<PER>', '<TICKER>')
data22.printSchema()

root
 |-- <DATE>: integer (nullable = true)
 |-- <TIME>: integer (nullable = true)
 |-- <OPEN>: double (nullable = true)
 |-- <HIGH>: double (nullable = true)
 |-- <LOW>: double (nullable = true)
 |-- <CLOSE>: double (nullable = true)
 |-- <VOL>: integer (nullable = true)



In [4]:
data22.show(10)

+--------+------+-------+-------+-------+-------+-------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>|  <VOL>|
+--------+------+-------+-------+-------+-------+-------+
|20220103| 70100|74.6775|74.8375|74.6625|  74.79|2429000|
|20220103| 70200|  74.81|  74.81| 74.755|74.7875| 876000|
|20220103| 70300|74.7875| 74.875|74.7875| 74.875|1592000|
|20220103| 70400|74.8475|74.8625|74.8425|74.8475|1208000|
|20220103| 70500|74.8425| 74.875|74.8275|74.8575| 426000|
|20220103| 70600|74.8625|  74.89|74.8625|  74.89| 574000|
|20220103| 70700|  74.89| 74.895|  74.89| 74.895|  14000|
|20220103| 70800| 74.895| 74.915|  74.89| 74.905| 767000|
|20220103| 70900|74.8825|74.8975| 74.865| 74.865|1103000|
|20220103| 71000| 74.885|  74.95| 74.885|  74.93|2011000|
+--------+------+-------+-------+-------+-------+-------+
only showing top 10 rows



In [5]:
data22.summary().pandas_api()

,summary,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,count,194619,194619,194619,194619,194619,194619,194619
1,mean,2.0222016135058757E7,139444.36206125814,69.27464305900287,69.30530000924762,69.24701700502102,69.27504194091888,2716198.870613866
2,stddev,3358.8902809414512,36029.872361950474,11.785004148386559,11.805320739239514,11.771317213486764,11.784747173346558,3786015.5677051907
3,min,20220103,70100,50.14,50.245,50.0125,50.1325,1000
4,25%,20220323,112000,60.6,60.6175,60.5875,60.6,531000
5,50%,20220803,140700,68.14,68.175,68.1125,68.14,1531000
6,75%,20221121,165300,75.56,75.58,75.5425,75.56,3416000
7,max,20230228,235000,121.1,121.5275,121.0,121.21,122028000


In [6]:
data22.where(F.col('<VOL>')<123000000).orderBy(F.col('<VOL>').desc()).show(10)

+--------+------+-------+-------+--------+-------+---------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|   <LOW>|<CLOSE>|    <VOL>|
+--------+------+-------+-------+--------+-------+---------+
|20220420|114300|77.3225| 77.475|  71.885| 75.675|122028000|
|20220214|154000| 76.715| 76.715|  76.185|76.3575|120135000|
|20220124|160000|  78.57|  78.93|   78.51|78.8625|115489000|
|20220214|154400|76.0575|   76.1|   75.93|76.0725|100043000|
|20220218|161000|75.8375|  76.06| 75.8375| 75.955| 84542000|
|20220214|154300|76.2625|   76.3| 76.0175|76.0575| 84020000|
|20220221|154500|  78.22|78.5125|   78.22|78.3925| 77397000|
|20221019|135200| 61.515| 61.635|   61.41|  61.42| 77189000|
|20220303|174200| 110.07|  110.5|109.8275|  110.0| 77032000|
|20220221|113600|76.9575|  77.26| 76.9575|77.1925| 75296000|
+--------+------+-------+-------+--------+-------+---------+
only showing top 10 rows



In [7]:
data22.where(F.col('<DATE>')==20220424).orderBy(F.col('<TIME>').desc()).show(5)

+------+------+------+------+-----+-------+-----+
|<DATE>|<TIME>|<OPEN>|<HIGH>|<LOW>|<CLOSE>|<VOL>|
+------+------+------+------+-----+-------+-----+
+------+------+------+------+-----+-------+-----+



In [8]:
date22 = (data22
    .groupBy('<DATE>')
    .agg(F.count('<DATE>').alias('Count'))
    #.orderBy(F.col('Count').desc())
)
date22.show(10)

date22.select('Count').distinct().count()

+--------+-----+
|  <DATE>|Count|
+--------+-----+
|20220426|  541|
|20221215|  715|
|20220103| 1006|
|20220922|  707|
|20230113|  704|
|20220207| 1008|
|20220701|  541|
|20220902|  541|
|20220617|  541|
|20220805|  541|
+--------+-----+
only showing top 10 rows



71

In [9]:
data22.select('<DATE>').distinct().count()

292

In [10]:
data22.where(F.col('<DATE>')==20220426).orderBy(F.col('<TIME>').desc()).show(5)
data22.where(F.col('<DATE>')==20220426).orderBy(F.col('<TIME>').asc()).show(5)

+--------+------+-------+-------+-------+-------+-------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>|  <VOL>|
+--------+------+-------+-------+-------+-------+-------+
|20220426|190000|73.2325| 73.525|  73.23| 73.525|6611000|
|20220426|185900|73.1825|73.2475|73.1825|  73.23|1776000|
|20220426|185800|  73.14|73.1825|  73.13|73.1825|3747000|
|20220426|185700|73.1175|73.1475|73.1175|  73.14|1166000|
|20220426|185600|  73.09|73.1175|  73.09|73.1175| 855000|
+--------+------+-------+-------+-------+-------+-------+
only showing top 5 rows

+--------+------+-------+------+-------+-------+--------+
|  <DATE>|<TIME>| <OPEN>|<HIGH>|  <LOW>|<CLOSE>|   <VOL>|
+--------+------+-------+------+-------+-------+--------+
|20220426|100000| 73.125|73.125| 73.125| 73.125|   98000|
|20220426|100100|  73.12|73.125| 72.225|72.4675|10365000|
|20220426|100200|  72.43| 72.49|  72.06|   72.3|12502000|
|20220426|100300|   72.3|72.815|  72.26| 72.535|16887000|
|20220426|100400|72.5275|72.535|72.2975| 72.365

In [11]:
data22.where(F.col('<DATE>')==20220103).orderBy(F.col('<TIME>').desc()).show(5)
data22.where(F.col('<DATE>')==20220103).orderBy(F.col('<TIME>').asc()).show(5)

+--------+------+-------+-------+-------+-------+------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>| <VOL>|
+--------+------+-------+-------+-------+-------+------+
|20220103|235000| 74.555|  74.56|74.5425|74.5425|652000|
|20220103|234900|  74.55|  74.56|  74.55|74.5525|236000|
|20220103|234800|  74.56|74.5625|  74.55|74.5625|391000|
|20220103|234700|74.5475|  74.56|74.5475|74.5475|203000|
|20220103|234600| 74.545|  74.55| 74.545|74.5475|122000|
+--------+------+-------+-------+-------+-------+------+
only showing top 5 rows

+--------+------+-------+-------+-------+-------+-------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>|  <VOL>|
+--------+------+-------+-------+-------+-------+-------+
|20220103| 70100|74.6775|74.8375|74.6625|  74.79|2429000|
|20220103| 70200|  74.81|  74.81| 74.755|74.7875| 876000|
|20220103| 70300|74.7875| 74.875|74.7875| 74.875|1592000|
|20220103| 70400|74.8475|74.8625|74.8425|74.8475|1208000|
|20220103| 70500|74.8425| 74.875|74.8275|74.8575| 426000

In [12]:
data23 = spark.read.csv(".//archive/minute_230301_231108.csv", sep=';', encoding="UTF-8", header=True, inferSchema=True)
data23.printSchema()

root
 |-- <TICKER>: string (nullable = true)
 |-- <PER>: integer (nullable = true)
 |-- <DATE>: integer (nullable = true)
 |-- <TIME>: integer (nullable = true)
 |-- <OPEN>: double (nullable = true)
 |-- <HIGH>: double (nullable = true)
 |-- <LOW>: double (nullable = true)
 |-- <CLOSE>: double (nullable = true)
 |-- <VOL>: integer (nullable = true)



In [13]:
data23 = data23.drop('<PER>', '<TICKER>')
data23.printSchema()
data23.show(10)

root
 |-- <DATE>: integer (nullable = true)
 |-- <TIME>: integer (nullable = true)
 |-- <OPEN>: double (nullable = true)
 |-- <HIGH>: double (nullable = true)
 |-- <LOW>: double (nullable = true)
 |-- <CLOSE>: double (nullable = true)
 |-- <VOL>: integer (nullable = true)

+--------+------+-------+-------+-------+-------+-------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>|  <VOL>|
+--------+------+-------+-------+-------+-------+-------+
|20230103| 70100|   70.8|70.9425|   70.0|70.8575|1259000|
|20230103| 70200|70.8475|70.8475|70.5125|70.7675| 631000|
|20230103| 70300| 70.565| 70.585| 70.515| 70.515|  97000|
|20230103| 70400| 70.515|70.7175| 70.515|70.6675|  64000|
|20230103| 70500|  70.62|  70.62|  70.54|70.5875|  22000|
|20230103| 70600| 70.535|  70.69| 70.535|70.5725| 202000|
|20230103| 70700| 70.665|70.6675|   70.6|70.6675|  57000|
|20230103| 70800|70.6025|70.6875|   70.6|   70.6| 210000|
|20230103| 70900|   70.6|   70.6|  70.52|  70.52|  17000|
|20230103| 71000|70.5575| 70.6

In [14]:
data23.summary().pandas_api()

,summary,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,count,149912,149912,149912,149912,149912,149912,149912
1,mean,2.023058680553925E7,130044.75892523614,84.73647846736571,84.75315646846016,84.72096484937829,84.73711667511606,1498470.5026949144
2,stddev,296.73865757754123,33769.16969853376,9.718493270762863,9.721185372542067,9.7160694764142,9.718580233836937,2158623.1220761454
3,min,20230103,70100,67.0975,67.17,67.0875,67.0975,1000
4,25%,20230321,102100,76.5325,76.545,76.52,76.535,290000
5,50%,20230608,131400,82.4125,82.4225,82.3925,82.4075,859000
6,75%,20230824,160700,94.125,94.15,94.11,94.1275,1884000
7,max,20231108,190000,102.2925,102.345,102.225,102.3125,135243000


In [15]:
data23.where(F.col('<VOL>')<140000000).orderBy(F.col('<VOL>').desc()).show(10) #посмотреть не выброс ли по столбцу объема оборота

+--------+------+-------+-------+-------+-------+---------+
|  <DATE>|<TIME>| <OPEN>| <HIGH>|  <LOW>|<CLOSE>|    <VOL>|
+--------+------+-------+-------+-------+-------+---------+
|20230316|122600| 76.205| 77.315| 76.205|   77.3|135243000|
|20230316|123000|  76.58|  76.64|   76.0|  76.17|106379000|
|20230825|173100|  95.41|95.4525|95.2275|  95.29|100977000|
|20230316|122700| 77.325|  77.49|77.0575|77.1375| 71010000|
|20230404|151100| 79.255|  79.26| 79.095|  79.11| 62870000|
|20230406|115600|80.4575|80.5025|80.4575|   80.5| 58359000|
|20230316|122900|76.9075| 76.915|76.5225|76.5925| 58287000|
|20230524|160100|  80.11|80.2075|  80.11|80.1925| 49317000|
|20230316|122800|77.1375|  77.15|76.8925|76.8925| 46315000|
|20231103|110100|93.1275|93.1275|  92.45| 92.685| 43350000|
+--------+------+-------+-------+-------+-------+---------+
only showing top 10 rows



In [16]:
date23 = (data23
    .groupBy('<DATE>')
    .agg(F.count('<DATE>').alias('Count'))
    #.orderBy(F.col('Count').desc())
)
date23.show(10)

date23.select('Count').distinct().count() #число количества уникальных точек в день

+--------+-----+
|  <DATE>|Count|
+--------+-----+
|20230530|  700|
|20230113|  704|
|20230925|  679|
|20230626|  720|
|20230111|  699|
|20230301|  690|
|20230310|  678|
|20230105|  673|
|20230825|  682|
|20231103|  701|
+--------+-----+
only showing top 10 rows



58

In [17]:
data23.select('<DATE>').distinct().count() #уникальных дней

217